In [1]:
def initial_pressure(upper_pressure,lower_pressure):
    return np.linspace(lower_pressure,upper_pressure,number_of_points)

In [2]:
def build_system_matrix(Cv_top,Cv_bottom):
    #build system matrix
    A = np.diag([-2.]*number_of_points,0) + np.diag([1.]*(number_of_points-1),1) + np.diag([1.]*(number_of_points-1),-1)
    #material properties bottom layer
    transition_index = int(number_of_points/2)
    A[:transition_index,:] *= Cv_bottom * dt/dz**2
    #material properties top layer
    A[transition_index+1:,:] *= Cv_top * dt/dz**2
    #Transition condition (flux continuity)
    A[transition_index,transition_index-1] = Cv_bottom
    A[transition_index,transition_index+1] = Cv_top
    A[transition_index,transition_index] = -(Cv_bottom + Cv_top)
    A[transition_index,:] *= dt/dz**2
    #add previous time step
    A += np.diag([1.] * number_of_points,0)
    #upper
    A[-1,:] = 0.
    #lower
    A[0,0] = 1.
    return A

In [3]:
def run_time_steps(A,sol,T):
    for t in T:
        #step forward
        u = np.dot(A,sol[-1])
        #apply bc
        u[0] = u[1]
        sol = np.append(sol,[u],0)
    return sol

In [4]:
def find_closest(array,value):
    return (np.abs(array - value)).argmin()

In [6]:
def run_and_plot_simulation_CF(u_0,Cv_1,Cv_2,figname):
    solutions_12 = np.array([u_0])
    solutions_21 = np.array([u_0])
    solutions_11 = np.array([u_0])
    A_12 = build_system_matrix(Cv_1,Cv_2)
    A_21 = build_system_matrix(Cv_2,Cv_1)
    A_11 = build_system_matrix(Cv_1,Cv_1)

    with cf.ProcessPoolExecutor(max_workers=3) as executor:
        solutions_12 = executor.submit(run_time_steps, A_12, solutions_12, time)
        solutions_21 = executor.submit(run_time_steps, A_21, solutions_21, time)
        solutions_11 = executor.submit(run_time_steps, A_11, solutions_11, time)

    fig, ax = plt.subplots(ncols=3,figsize=(18,8))
    Ts = [0.0,0.01,0.02,0.1,0.2,0.5,0.8,1.0]
    for t in Ts:
        idx = find_closest(time,t)
        ax[0].plot(solutions_11.result()[idx],zeta,label=r'$T_\mathrm{V} = %.2f$' %time[idx])
        ax[1].plot(solutions_12.result()[idx],zeta)
        ax[2].plot(solutions_21.result()[idx],zeta)

    for i in range(3):
        ax[i].set_xlabel(r'$\Delta u\ /\ p_0$')
        ax[i].set_ylabel(r'$z\ /\ H$')

    ax[0].set_title(r'Ton, homogen')
    ax[1].set_title(r'Ton auf Schluff')
    ax[2].set_title(r'Schluff auf Ton')
    fig.legend(ncol=8,loc='upper center')
    fig.savefig(figname)

In [ ]:
def run_and_plot_simulation(u_0,Cv_1,Cv_2,figname):
    solutions_12 = np.array([u_0])
    solutions_21 = np.array([u_0])
    solutions_11 = np.array([u_0])
    A_12 = build_system_matrix(Cv_1,Cv_2)
    A_21 = build_system_matrix(Cv_2,Cv_1)
    A_11 = build_system_matrix(Cv_1,Cv_1)
    solutions_12 = run_time_steps(A_12, solutions_12, time)
    solutions_21 = run_time_steps(A_21, solutions_21, time)
    solutions_11 = run_time_steps(A_11, solutions_11, time)

    fig, ax = plt.subplots(ncols=3,figsize=(18,8))
    Ts = [0.0,0.01,0.02,0.1,0.2,0.5,0.8,1.0]
    for t in Ts:
        idx = find_closest(time,t)
        ax[0].plot(solutions_11[idx],zeta,label=r'$T_\mathrm{V} = %.2f$' %time[idx])
        ax[1].plot(solutions_12[idx],zeta)
        ax[2].plot(solutions_21[idx],zeta)

    for i in range(3):
        ax[i].set_xlabel(r'$\Delta u\ /\ p_0$')
        ax[i].set_ylabel(r'$z\ /\ H$')

    ax[0].set_title(r'Ton, homogen')
    ax[1].set_title(r'Ton auf Schluff')
    ax[2].set_title(r'Schluff auf Ton')
    fig.legend(ncol=8,loc='upper center')
    fig.savefig(figname)